In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,10.12,69,100,5.94,NZ,1697555085
1,1,porto uniao,-26.2381,-51.0783,22.47,84,100,0.99,BR,1697555464
2,2,blackmans bay,-43.0167,147.3167,8.07,84,38,2.19,AU,1697555464
3,3,puerto ayora,-0.7393,-90.3518,28.98,89,96,3.13,EC,1697555116
4,4,edinburgh of the seven seas,-37.0676,-12.3116,12.47,68,100,4.16,SH,1697555100


In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp_s = city_data_df["Max Temp"] > 21
max_temp_s = city_data_df["Max Temp"] < 27
wind_speed_s = city_data_df["Wind Speed"] > 4.5
city_temp_range_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]

# Drop any rows with null values
clean_city_temp_range_df = city_temp_range_df.dropna()

# Display sample data
clean_city_temp_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,alofi,-19.0595,-169.9187,22.84,73,67,10.71,NU,1697555467
46,46,vila franca do campo,37.7167,-25.4333,22.23,71,70,4.68,PT,1697555196
50,50,ribeira grande,38.5167,-28.7000,22.22,78,75,6.17,PT,1697555478
57,57,afaahiti,-17.7500,-149.2833,22.93,85,83,7.02,PF,1697555112
63,63,merida,20.9667,-89.6167,26.68,61,75,7.60,MX,1697555481
78,78,west island,-12.1568,96.8225,25.99,89,75,7.20,CC,1697555045
79,79,celestun,20.8667,-90.4000,26.32,57,50,6.04,MX,1697555485
104,104,praia da vitoria,38.7333,-27.0667,24.14,64,20,5.19,PT,1697555030
112,112,carnarvon,-24.8667,113.6333,23.83,53,0,9.31,AU,1697555182
119,119,ta`u,-14.2336,-169.5144,25.68,83,86,8.09,AS,1697555495


In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
13,13,alofi,-19.0595,-169.9187,22.84,73,67,10.71,NU,1697555467,
46,46,vila franca do campo,37.7167,-25.4333,22.23,71,70,4.68,PT,1697555196,
50,50,ribeira grande,38.5167,-28.7000,22.22,78,75,6.17,PT,1697555478,
57,57,afaahiti,-17.7500,-149.2833,22.93,85,83,7.02,PF,1697555112,
63,63,merida,20.9667,-89.6167,26.68,61,75,7.60,MX,1697555481,
78,78,west island,-12.1568,96.8225,25.99,89,75,7.20,CC,1697555045,
79,79,celestun,20.8667,-90.4000,26.32,57,50,6.04,MX,1697555485,
104,104,praia da vitoria,38.7333,-27.0667,24.14,64,20,5.19,PT,1697555030,
112,112,carnarvon,-24.8667,113.6333,23.83,53,0,9.31,AU,1697555182,
119,119,ta`u,-14.2336,-169.5144,25.68,83,86,8.09,AS,1697555495,


In [12]:
# Set parameters to search for a hotel
radius = 1000
limit = 1
categories = "accommodation.hotel"
params = {
     "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alofi - nearest hotel: No hotel found
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
ribeira grande - nearest hotel: No hotel found
afaahiti - nearest hotel: No hotel found
merida - nearest hotel: Hotel Mejorada Mérida
west island - nearest hotel: No hotel found
celestun - nearest hotel: Hotel Flamingo Playa
praia da vitoria - nearest hotel: Salles
carnarvon - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
hirara - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
tolanaro - nearest hotel: Hôtel Mahavokey
neiafu - nearest hotel: Port Wine Guest House
tual - nearest hotel: No hotel found
villa regina - nearest hotel: Nuevo Cheo
yeppoon - nearest hotel: No hotel found
namibe - nearest hotel: Hotel Chik Chik Namibe
limuru - nearest hotel: The Limuru Gardens
nuku'alofa - nearest hotel: City Hotel
papao - nearest hotel: No hotel found
neon karlovasion - nearest hotel: Ξενοδοχ

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
13,13,alofi,-19.0595,-169.9187,22.84,73,67,10.71,NU,1697555467,No hotel found
46,46,vila franca do campo,37.7167,-25.4333,22.23,71,70,4.68,PT,1697555196,Vinha d'Areia Beach Hotel
50,50,ribeira grande,38.5167,-28.7000,22.22,78,75,6.17,PT,1697555478,No hotel found
57,57,afaahiti,-17.7500,-149.2833,22.93,85,83,7.02,PF,1697555112,No hotel found
63,63,merida,20.9667,-89.6167,26.68,61,75,7.60,MX,1697555481,Hotel Mejorada Mérida
78,78,west island,-12.1568,96.8225,25.99,89,75,7.20,CC,1697555045,No hotel found
79,79,celestun,20.8667,-90.4000,26.32,57,50,6.04,MX,1697555485,Hotel Flamingo Playa
104,104,praia da vitoria,38.7333,-27.0667,24.14,64,20,5.19,PT,1697555030,Salles
112,112,carnarvon,-24.8667,113.6333,23.83,53,0,9.31,AU,1697555182,No hotel found
119,119,ta`u,-14.2336,-169.5144,25.68,83,86,8.09,AS,1697555495,No hotel found


In [13]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)